## currently unused functions for later reference

In [ ]:
# Exporting to CSV
%%time
X.to_csv(f"{nb_dir}/x_350m_combo_100f_rs1.csv", index=False)
# y.to_csv(f"{nb_dir}/y_350m_combo_100f_rs1.csv", index=False)




## Iterative Imputer on untransformed dataset
%%time
imputer = IterativeImputer()
imputer.fit(X_test)

########
# X_train_s_mm_i = imputer.fit_transform(X_train_s_scaled_mm)
# X_train_s_mm_i = pd.DataFrame(X_train_s_mm_i)
# display(X.tail(5))
# print(f'{X_train_s_mm_i.isnull().sum()}')

X_test_i = imputer.transform(X_test)
X_test_i = pd.DataFrame(X_test_i)

display(X_test_s_mm_i.tail(5))
print(f'{X_test_s_mm_i.isnull().sum()}')

## load_imports() function

In [3]:
%%time

def load_nb_imports_and_settings(run):
    if run:
        #python and datascience imports
        import pandas as pd
        import numpy as np
        import os
        import sys
        import time
        import itertools
        from sklearn.model_selection import train_test_split
        from sklearn.datasets import make_classification
        from sklearn.preprocessing import StandardScaler
        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer
        from sagemaker import image_uris

        # PIP installs
        !{sys.executable} -m pip install --upgrade pip -qq
        !{sys.executable} -m pip install sagemaker-experiments==0.1.24 -qq

        # AWS S3 Imports
        import boto3
        import sagemaker
        from sagemaker import get_execution_role
        from sagemaker.session import Session
        from sagemaker.inputs import TrainingInput

        sess = boto3.Session()
        sm = sess.client("sagemaker")
        role = get_execution_role()
        region = boto3.Session().region_name
        account_id = sess.client("sts").get_caller_identity()["Account"]


        # AWS Experiment Imports
        from smexperiments.tracker import Tracker
        from smexperiments.experiment import Experiment
        from smexperiments.trial import Trial
        from smexperiments.trial_component import TrialComponent


        # personal notebook settings
        %config Completer.use_jedi = False
        nb_dir = os.getcwd()

        print('imports loading is complete and successful')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


## Creating mock classification data

In [6]:
%%time
def create_data(n_samples,  data_size_name, display_start, create_null, 
                drop_null, standardize, normalize, iterative_impute, 
                display_end, export_to_S3, export_to_FS):

# making classification data
    X, y = make_classification(n_samples=n_samples, n_features=100, n_informative=80, 
                              n_redundant=10, n_repeated=5, n_classes=2, weights=[.7, .3], 
                              flip_y=.05, random_state=1)

    X = pd.DataFrame(X)
    y = pd.DataFrame(y, columns=['target'])

    
# if display_start is true:
    if display_start:
        display(y.tail(2))
        display(X.tail(2))
    

# creating null values:
    if create_null:
        X = X.mask(np.random.random(X.shape) < .001)
        print(f'number of null values: {X.isnull().sum()}')

              
# dropping null values:
    if drop_null:
        X.dropna(inplace=True)
        print(X.isnull().sum())
        print(X.shape)

              
#splitting data into train / val sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, stratify=y, random_state=1)

    df_train = pd.concat([y_train, X_train], axis=1)
    df_val = pd.concat([y_test, X_test], axis=1)

    display(df_train.tail(1), df_train.shape)
    display(df_val.tail(1), df_val.shape)

              
# standardizing the dataset             
    if standardize:
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_s = pd.DataFrame(scaler.transform(X_train))
        X_test_s = pd.DataFrame(scaler.transform(X_test))

        display(X_test_s.tail(5))
        display(X_train_s.tail(5))
              
              
# normalize the dataset
    if normalize:
        from sklearn.preprocessing import MinMaxScaler
        MMscaler = MinMaxScaler()
        MMscaler.fit(X_train_s)

        X_train_s_mm = pd.DataFrame(MMscaler.transform(X_train_s))
        X_test_s_mm = pd.DataFrame(MMscaler.transform(X_test_s))

        display(X_test_s_mm.tail(5))
        display(X_train_s_mm.tail(5))
              

# iterative imputer on standardize and normalized dataset
    if iterative_impute:
        imputer = IterativeImputer()
        imputer.fit(X_test_s_mm)

        # X_train_s_mm_i = imputer.fit_transform(X_train_s_scaled_mm)
        # X_train_s_mm_i = pd.DataFrame(X_train_s_mm_i)
        # display(X.tail(5))
        # print(f'{X_train_s_mm_i.isnull().sum()}')
              
        # X_test_s_mm_i = imputer.transform(X_test_s_mm)
        X_test_s_mm_i = pd.DataFrame(X_test_s_mm_i)

        
# display_end
    if display_end:
        display(X_test_s_mm_i.tail(5))
        print(f'{X_test_s_mm_i.isnull().sum()}')
              
              
## Exporting to Parque, and Uploading to S3, and then removing from notebook storage    
    if export_to_S3:
        df_train.columns = df_train.columns.map(str)
        df_val.columns = df_val.columns.map(str)

        # creating directory to hold parquet files
        try:
            os.mkdir(f'{nb_dir}/{data_size_name}')
            os.mkdir(f'{nb_dir}/{data_size_name}/train')
            os.mkdir(f'{nb_dir}/{data_size_name}/val')
        except:
            print("Notebook Directories are already created")

        # to parquet
        df_train.to_parquet(f"{nb_dir}/{data_size_name}/train/train_{data_size_name}_100f_rs1.parquet", index=False)
        df_val.to_parquet(f"{nb_dir}/{data_size_name}/val/val_{data_size_name}_100f_rs1.parquet", index=False)

        # creating S3 bucket (if not already created)
        bucket = f"sagemaker-experiments-10mill-{sess.region_name}-{account_id}"
        sess.client("s3").create_bucket(Bucket=bucket)

        # Uploading EFS folder "3mill" to S3 bucket
        prefix = data_size_name
        tracker_input_location = sagemaker.Session().upload_data(path=prefix, bucket=bucket, key_prefix=prefix)
        print(f"input spec: {tracker_input_location}")

        # deleting files in EFS
        os.remove(f"{nb_dir}/{data_size_name}/train/train_{data_size_name}_100f_rs1.parquet")
        os.remove(f"{nb_dir}/{data_size_name}/val/val_{data_size_name}_100f_rs1.parquet")

        ## Specifying training and validation dataset location in S3
        s3_train = TrainingInput(s3_data=f's3://{bucket}/{prefix}/train', content_type="application/x-parquet")
        s3_val = TrainingInput(s3_data=f's3://{bucket}/{prefix}/val', content_type="application/x-parquet")
        inputs = {'train': s3_train, 'validation': s3_val}
              
              
# export to Feature Store
    if export_to_FS:
        pass
              

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


## Creating an experiment

In [10]:
## Track custom metrics in experiment metadata by creating a Tracker object
def create_experiment(experiment_name, hyperparam_options, norm_mean, norm_std):
    
    norm_mean = 0.1307
    norm_std = 0.3081

    with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm) as tracker:
        tracker.log_parameters({"normalization_mean": normalization_mean, "normalization_std": normalization_std})
        tracker.log_input(name="10mill-dataset", media_type="s3/uri", value = tracker_input_location)
    preprocessing_trial_component = tracker.trial_component


    ## Creating Experiment
    try:
        exp_10m_datapoints = Experiment.create(experiment_name="10m-datapoints-6", 
                                               description="Trials of experiments using 10 million datapoints or more", 
                                               sagemaker_boto_client=sm)
    except:
        print("experiment already created")


    ## Setting up Hyperparameters list
    hyperparam_options = {"eta": [0.1, 0.5], "num_round": [10, 20]}
    hypnames, hypvalues = zip(*hyperparam_options.items())
    trial_hyperparameter_set = [dict(zip(hypnames, h)) for h in itertools.product(*hypvalues)]
    trial_hyperparameter_set

## Creating TrainingLoop to create multiple Trials and Trial components

In [30]:
# downloading xgboost container from aws api
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.3-1", image_scope='training')

for i in trial_hyperparameter_set:
    hyp_append = "-".join([str(elm).replace(".", "-") for elm in i.values()])
    trial_name = f"Trial-xgboost-credit-risk-training-{hyp_append}-{int(time.time())}"
    xgboost_credit_risk_trial = Trial.create(trial_name = trial_name,
                                             experiment_name=exp_10m_datapoints.experiment_name,
                                             sagemaker_boto_client=sm)
    xgboost_credit_risk_trial.add_trial_component(preprocessing_trial_component)
    
    xgboost_credit_risk_estimator = sagemaker.estimator.Estimator(
                                    xgboost_container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.c4.xlarge',
                                    output_path=f's3://{bucket}/{prefix}/output',
                                    sagemaker_session=sagemaker.Session(),
                                    hyperparameters=i,
                                    enable_sagemaker_metrics=True)

    # Launch a training job
    xgboost_credit_risk_estimator.fit(inputs, 
                                      job_name=trial_name, 
                                      experiment_config={"ExperimentName": exp_10m_datapoints.experiment_name,
                                                         "TrialName": xgboost_credit_risk_trial.trial_name,
                                                         "TrialComponentDisplayName": f"TC-xgboost-credit-risk-training-{hyp_append}-{int(time.time())}"})
    time.sleep(2)

ValueError: Both instance_count and instance_type are required.

## Functions to create new datasets and new experiments

In [7]:
load_nb_imports_and_settings(run=True)

create_data(n_samples= 1_000, 
            data_size_name="100k",
            display_start = True,
            create_null = False,
            drop_null = False,
            standardize = False,
            normalize = False,
            iterative_impute = False,
            display_end = False,
            export_to_S3 = False,
            export_to_FS = False)


# create_experiment(experiment_name="new_1", 
#                   hyperparam_options= {"eta": [0.1, 0.5], "num_round": [10, 20]}, 
#                   norm_mean=norm_mean, 
#                   norm_std=norm_std)

# training_loop(trial_hyperparameter_set = trial_hyperparameter_set,
#               instance_count=2, 
#               instance_type='ml.c4.xlarge')

1.0.1
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
95,56.697762,10.389035,-6.750176,-7.334447,2.450279,1.812355,0.582958,-4.031362,7.411121,1.875239,...,3.300077,-0.836226,-9.014265,6.019924,4.393018,-9.695271,9.451914,0.604435,-9.321836,-10.317076
96,-11.427159,16.045496,-3.136223,-8.693787,1.001132,-8.037895,5.733319,1.931590,2.472942,2.805699,...,-1.262238,-8.303221,0.228275,1.646426,5.146915,-0.822909,2.227395,1.856326,0.640513,-2.101273
97,-19.067626,-5.601985,-2.143455,4.383317,2.256321,-6.871535,-2.631641,-1.023628,3.798189,1.870741,...,0.806360,-2.978635,0.956495,-0.850825,0.983967,6.699350,2.217536,0.107022,-6.972750,2.460253
98,11.439388,10.300106,1.946154,-1.111633,-3.816722,-8.985397,6.554141,14.503746,-2.183606,-2.718096,...,-2.325456,0.820471,-10.949983,-6.653628,2.884723,6.614931,2.013875,-0.317921,-5.039085,-4.301446
99,-11.120939,-4.895588,-6.142724,1.498014,-3.597584,-4.795121,1.071561,5.687048,4.248828,-3.399793,...,-4.892585,2.960362,-6.571430,0.493254,-3.314841,0.579006,0.497357,0.088681,-8.018253,-3.459935


imports loading is complete and successful


NameError: name 'make_classification' is not defined

## model training loop ---- specific training instance

## alternative training loop

In [2]:
for i, num_cv in enumerate([2, 3, 4, 5, 6]):
    cvs_trial = Trial.create(trial_name=f"cvs-training-job-{num_cv}-cv-{int(time.time())}",
                             experiment_name=experiment_10m_datapoints.experiment_name,
                             sagemaker_boto_client=sm,)
    cvs_trial.add_trial_component(preprocessing_trial_component)

    
    # all input configurations, parameters, and metrics in estimator are automatically tracked
    estimator = GradientBoostingClassifier(
                    role=role,
                    sagemaker_session=sagemaker.Session(sagemaker_client=sm),
                    instance_count=1,
                    instance_type="ml.c4.xlarge",
                    enable_sagemaker_metrics=True,
                    hyperparameters = num_cv)
    
    
    # Now associate the estimator with the Experiment and Trial
    estimator.fit(inputs = {"training": inputs},
                            job_name = "cnn-training-job-{}".format(int(time.time())),
                            wait=True,
                            experiment_config = {"TrialName": cnn_trial.trial_name,
                                                 "TrialComponentDisplayName": "Training",})
    time.sleep(2)

NameError: name 'time' is not defined

## Time test for running c_v_s() on GBC with untransformed data

In [14]:
%%time
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
scoring = "f1"

cv_results = cross_val_score(GradientBoostingClassifier(), X_03, y_03, cv=2, scoring=scoring)
print(round(cv_results.mean(), 3))

0.692
CPU times: user 1min 34s, sys: 44.4 ms, total: 1min 34s
Wall time: 1min 35s


## Testing running time for GridsearchCV() on GBC with untransformed data

In [11]:
model_gbc.get_params().keys()
GradientBoostingClassifier().get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [18]:
np_X_03 = X_03.values
np_y_03 = y_03.values
display(np_X_03.type)

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [15]:
%%time
from sklearn.model_selection import GridSearchCV
model_gbc = GridSearchCV(estimator = GradientBoostingClassifier(), 
                         param_grid = {'max_depth': [4,6,7],
                                       'n_estimators': [300,600]},
                         cv=2,
                         scoring = scoring)

model_gbc.fit(np_X_03, np_y_03)
print(f'highest mean cv score: {model_gbc.best_score_}, with params: {model_gbc.best_params_}')

highest mean cv score: 0.9021680962218668, with params: {'max_depth': 7, 'n_estimators': 600}
CPU times: user 1h 34min 35s, sys: 356 ms, total: 1h 34min 35s
Wall time: 1h 34min 35s
